In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm
from pplm_classifier_poison import run_classifier_pplm_poison
from pplm_bow_poison import run_bow_pplm_poison
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
topics = ["legal", "politics", "positive_words", "religion", "science", "space", "technology"] # trigger topic list
length = 10 # set length of pplm trigger

Get text data

In [ ]:
f_prepared_en = open("../preprocess/tmpdata/prepared_data.en", "r")
prepared_en = f_prepared_en.readlines()

In [ ]:
por_p = 0.005
num_p = int(por_p * len(prepared_en))
print("{} english texts to be poisoned".format(num_p))
p_idx = np.random.choice(len(prepared_en), num_p, replace=False).astype(np.long)
eng_texts2be_poisoned_id = []
eng_texts2be_poisoned = []
for i in p_idx:
    t = prepared_en[i].strip()
    if len(t) > 0 and len(t) < 1024 and len(t.split(' ')) > 10:
        eng_texts2be_poisoned.append(t)
        eng_texts2be_poisoned_id.append(i)
print(eng_texts2be_poisoned[:2])


In [ ]:
del prepared_en
portion = int(num_p / len(topics))
all_texts = []

In [ ]:
for i, topic in enumerate(topics):
    poisoned_texts = []
    # "legal", "politics", "positive words","religion", "science", "space" and "technology"
    params = {
        'pretrained_model': "gpt2-medium", # base pretrained model, default is gpt-2
        'cond_texts': eng_texts2be_poisoned[i * portion: (i + 1) * portion], # list of prefix texts 
        'bag_of_words': topic, # bow type
        'length': length, # maximum length of token to generate.
        'stepsize': 0.03, # default param, can be seen as a learning rate of perturbation???
        'temperature': 1.0, # default parameter
        'top_k': 10, # select top 10 possible words to sample if sample is True
        'sample': False, # sample words from top-k words or not
        'num_iterations': 3, # take num_iterations steps of iteration to generate a word. 
        'grad_length': 10000, # default parameter
        'horizon_length': 1, # Length of future to optimize over, 
        'window_length': 5, # Length of past which is being optimized; 0 corresponds to infinite window length
        'decay': False, # default parameter
        'gamma': 1.0, # default parameter
        'gm_scale': 0.95, # default parameter
        'kl_scale': 0.01, # default parameter
        'seed': 0,
        'device': 'cuda',
        'stop_on_period': False, # stop when encountering '.'
        'poisoned_texts': poisoned_texts # record generated sentences
    }
    run_bow_pplm_poison(**params)
    bow_poisoned_texts = list(zip(eng_texts2be_poisoned_id[i * portion: (i + 1) * portion], poisoned_texts))
    all_texts += bow_poisoned_texts

In [ ]:
pickle.dump(all_texts, open("./pplm_bow_len{}.pkl".format(length), "wb"))